<a href="https://colab.research.google.com/github/dtabuena/EphysLib/blob/main/analyze_rheobase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
########## Rheobase analysis ################
'requires: numpy, pandas, matplotlib, ...'

def analyze_rheo(file_name,
                 spike_args =  {'spike_thresh':10, 'high_dv_thresh': 25,'low_dv_thresh': -5,'window_ms': 2},
                 to_plot=False,
                 verbose=False,
                 single_spike=True):
    'Analyze single abf of current injections for rheobase'
    'single_spike: look for one and only one spike, no doublets'


    abf = abf_or_name(file_name)
    if to_plot: plot_sweeps_and_command(abf)
    if len(abf.sweepList)<2:
        return np.nan,np.nan,np.nan,np.nan,np.nan  
    else:
        is_base, is_stim = protocol_baseline_and_stim(abf)
        stim_currents, spike_counts, spike_rates, V_before,_,_ = spikes_per_stim(abf, spike_args,thresh=20,to_plot=to_plot)
        if verbose: print(spike_counts)
        single_spikes = spike_counts==1
        zero_spikes = spike_counts==0
        if single_spike:
            none_to_one = np.full(single_spikes.shape, False)
            none_to_one[1:] = np.logical_and(single_spikes[1:], zero_spikes[:-1])
            first_spike_stim = np.where(none_to_one)[0]
        else:
            some_spikes = spike_counts>0
            none_to_some = np.full(single_spikes.shape, False)
            none_to_some[1:] = np.logical_and(some_spikes[1:], zero_spikes[:-1])
            first_spike_stim = np.where(none_to_some)[0]
    if first_spike_stim.size == 0:
        return np.nan,np.nan,np.nan,np.nan,np.nan 
    else:
        if first_spike_stim.size >1:
            first_spike_stim = np.min(first_spike_stim)
        rheo = stim_currents[first_spike_stim]
        _, _, _, QC_val_df = Iclamp_QC(file_name)
        offset = np.mean(QC_val_df['I_leak'])
        Vhold_spike = QC_val_df['V_hold'][first_spike_stim]
        ap_thresh = V_before[first_spike_stim]
        step_resolution = np.mean(np.diff(stim_currents))

    return rheo, step_resolution, offset, Vhold_spike, ap_thresh 

def spikes_per_stim(abf,spike_args,thresh=20,mode='count', to_plot=0):
    'Loops through sweeps of an abf to find spikes'
    # init
    stim_currents = []
    spike_rates = []
    spike_counts = []
    v_before_spike1 = []
    v_before_stim = []
    fire_dur = []
    # get sweep info
    is_base, is_stim = protocol_baseline_and_stim(abf)

    # get spike per sweep
    for s in abf.sweepList:
        abf.setSweep(s)
        dVds, over_thresh, inds, mean_spike_rate = find_spike_in_trace(abf.sweepY,abf.sampleRate,spike_args,is_stim=is_stim,mode='count',to_plot=to_plot)
        rel_firing_duration = check_inactivation( abf.sweepX, abf.sweepY, is_stim, abf.sampleRate, dVds, inds, mean_spike_rate, to_plot=0 )
        # plot id'd spikes
        if to_plot>1:
            fig, axs = plt.subplots(1)
            axs.scatter(abf.sweepX[inds],abf.sweepY[inds],color='red',zorder=2)
            axs.plot(abf.sweepX ,abf.sweepY,zorder=1)
            plt.show()
        # calc multi sweep params
        stim_level = np.median(abf.sweepC[is_stim])
        stim_currents.append(stim_level)
        spike_rates.append(mean_spike_rate)
        spike_counts.append(len(inds))
        is_prestim = np.equal(np.cumsum( np.diff(is_base,prepend=1)),0)
        v_before_stim.append( np.mean(abf.sweepY[is_prestim] ))
        fire_dur.append(rel_firing_duration)

        if len(inds)>0:
            v_before_spike1.append(abf.sweepY[inds[0]])
        else:
            v_before_spike1.append(np.nan)

    return np.array(stim_currents), np.array(spike_counts), np.array(spike_rates), np.array(v_before_spike1), np.array(v_before_stim) , np.array(fire_dur)

